<a href="https://colab.research.google.com/github/lmastalerz/sutainability/blob/main/FaaS_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math 

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 200)

In [ ]:
# How many minutes Function App is cached after execution before resourtces 
# are deallocated 
FUN_TTL_MIN = 10

# This is the percentage of CPU that I'm assumming function consumes when it's running
# TODO: check how different values impact final results 
ACTIVE_FUNCTION_CPU_PCT = 0

# Datasets don't seem to match ideally, this is the amount of data across 
# invocation and duration datasets that's allowed to be missing 
MAX_MISSING_PCT = 2

In [ ]:
# Source files downloaded from here: https://azurecloudpublicdataset2.blob.core.windows.net/azurepublicdatasetv2/azurefunctions_dataset2019/azurefunctions-dataset2019.tar.xz


In [ ]:
# Function execution duration in miliseconds 
# Details of file structure here: 
# https://github.com/Azure/AzurePublicDataset/blob/master/AzureFunctionsDataset2019.md#function-execution-duration
dur_1 = pd.read_csv('/content/function_durations_percentiles.anon.d01.csv')
dur_1.head()

,HashOwner,HashApp,HashFunction,Average,Count,Minimum,Maximum,percentile_Average_0,percentile_Average_1,percentile_Average_25,percentile_Average_50,percentile_Average_75,percentile_Average_99,percentile_Average_100
0,5640c1597ef75fa9a7e9c6925022a039a4ba982924170953a51cfe4ef3a54ee9,5126901eff078c9a1f5295c859c9327588284a43cc2c0b4b261426b009ddfe6e,c968871b4ef0123401975d026b85cae2ad7dad9d06ae94f13b978bd66dfd063f,100,2880,0.000,2595.000,0,0,0,1,1,1376,2595
1,5640c1597ef75fa9a7e9c6925022a039a4ba982924170953a51cfe4ef3a54ee9,5126901eff078c9a1f5295c859c9327588284a43cc2c0b4b261426b009ddfe6e,1bc2d86badd21b18a8533d8e961e52585e5ad1fee0e2c2885220a8463a39492d,202,365,0.000,2596.000,0,0,1,1,6,2326,2596
2,41d6e09d0f86f5aaa1df842c1ac4f14fbd4dc676bec7e0f42b778b1d58f58c9a,77a93348150f5281c32c9027870a77983ad6eba72e6c2835fe2d27879dfc0753,b4d7cb9985cd8c1b7fe5ef888bbde1883929e9a5b5d2a65348b00ded9e2f8033,11,37815,1.000,3913.000,1,1,3,5,8,155,3146
3,e117b8c1676e11d859824b18fc0202ae8eaa89cb3f58e27837365ad7816a362d,5c84cc1fbc4261f22d6f626304670979c1f1b94709acc80397c73ab9c0b49959,ff612aae7b380ef81ccef063ff814354ee16018af65dc1bdf9437a09c41c3864,0,34466,0.000,123.000,0,0,0,0,0,1,10
4,3c8c13c6bd162490dae4402bcb9ff1e5c6c14bb21da35a443226a1fe61b63287,3b80dfff065220947f7d626dd6c9176731d3e17591e4968ed29c52bbb4d8894a,ee79840da82525548358f4b91d9fa1e550ec7af36bbe3e61c9a98e3be0fde824,389,288,78.000,5607.000,78,78,109,136,180,4761,5607


In [ ]:
# Grouping dusations by application - this is how resources are managed
# Taking 50th percentile per function and then (rather conservatively) 
# max across all functions within given app
# There has been multiple functions per app so need to group them by HashApp
dur_1.rename(columns={'percentile_Average_50':'duration_ms'}, inplace = True)
app_dur_1 = dur_1[['HashApp', 'Count', 'duration_ms']].groupby('HashApp').sum()
app_dur_1.head()

,Count,duration_ms
HashApp,,
000481d975e1672df56b11adc7957d13836870d14a1a5e0ab82a6970284b736a,7,3287
000b058a70dedf75e66071d3fe6f8701e31fed235942f4ae734e5d7737a3707f,396,6670
0016a719d18f743a52142f3b07efd6c17a034fa261e8f445d07da08a0b4bac72,50,955
0019d10aa8d3ef382c31a7b35d88b487d937d08dcdbdcd3867386807a4f8a8b0,182,1659
001bb40e4afa7a849199f0d584b4005e258b446259e7c066a81f9cd79568f9de,12,804


In [ ]:
# Number of function invocations 
# Details of file structure here:
# https://github.com/Azure/AzurePublicDataset/blob/master/AzureFunctionsDataset2019.md#function-invocation-counts
inv_1 = pd.read_csv('/content/invocations_per_function_md.anon.d01.csv')
inv_1.head()

,HashOwner,HashApp,HashFunction,Trigger,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,...,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440
0,71ca12c7af70d021e285b51b245942f8432df6463ff9f2c009b06b3f661f871f,7ca324d9fc836a5d4562811c11ce3719530ee919dd1fb91bcaf71942eab8240a,520dbd6bd906840012aa0c4b778743efc7c0ac7b7caf96b3d7f85d46209b7872,http,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
1,71ca12c7af70d021e285b51b245942f8432df6463ff9f2c009b06b3f661f871f,0d0ac65651f54ae3285a59564d64e39238b516fa1d5b565582032986e780b634,115ca7a2b5bc290052c3da74cd0347d19c3c67b7d5aa66e9a975e427f25fc7ed,http,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
2,71ca12c7af70d021e285b51b245942f8432df6463ff9f2c009b06b3f661f871f,a04487a6ba1e14296eb7647e4963180d28bef7a90a8fc5b3fbb894b8800418f3,93e6c664773bbec3a7f50a0e92fa7e97401a802dc6eed86ae062344eb0cb7c2e,orchestration,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,71ca12c7af70d021e285b51b245942f8432df6463ff9f2c009b06b3f661f871f,a04487a6ba1e14296eb7647e4963180d28bef7a90a8fc5b3fbb894b8800418f3,740c5c767e4b9978ee59a97d1829cfbaf755a47806a3114f0d4c182bb5a7e253,http,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,71ca12c7af70d021e285b51b245942f8432df6463ff9f2c009b06b3f661f871f,a04487a6ba1e14296eb7647e4963180d28bef7a90a8fc5b3fbb894b8800418f3,c108b4864b866b38b80d0e4594cc6d038f39668b804a1ba88d2b95d682a8ab20,http,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Grouping executions by application - this is how resources are managed
# New structure represents number of execution within one-minute buckets 
# for all functions running as a part of given App
# Automatically dropping all other columns like hash ID's and Trigger Type
app_inv_1 = inv_1.groupby('HashApp').sum()
app_inv_1.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,...,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440
HashApp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000481d975e1672df56b11adc7957d13836870d14a1a5e0ab82a6970284b736a,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
000b058a70dedf75e66071d3fe6f8701e31fed235942f4ae734e5d7737a3707f,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,2,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
0016a719d18f743a52142f3b07efd6c17a034fa261e8f445d07da08a0b4bac72,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,2,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
0019d10aa8d3ef382c31a7b35d88b487d937d08dcdbdcd3867386807a4f8a8b0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
001bb40e4afa7a849199f0d584b4005e258b446259e7c066a81f9cd79568f9de,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Joining data about invocations with data about duration 
app_1 = pd.merge(app_inv_1, app_dur_1, on='HashApp', how='outer')
app_1

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,...,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440,Count,duration_ms
HashApp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000481d975e1672df56b11adc7957d13836870d14a1a5e0ab82a6970284b736a,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,7.000,3287.000
000b058a70dedf75e66071d3fe6f8701e31fed235942f4ae734e5d7737a3707f,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,2.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,1.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,396.000,6670.000
0016a719d18f743a52142f3b07efd6c17a034fa261e8f445d07da08a0b4bac72,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0

In [ ]:
# Validate after joining to make sure not too much data is missing 
# Some observations have stats about duration but don't have stats about invocations 
# and vice versa - these are to be dropped unless they are above threshold 
no_inv_idx = np.isnan(app_1['1'])
no_dur_idx = np.isnan(app_1['duration_ms'])

missing_dur_pct = len(app_1[no_inv_idx]) / len(app_1.index) * 100 
missing_inv_pct = len(app_1[no_dur_idx]) / len(app_1.index) * 100

print('Missing durations: {:.2f}%'.format(missing_dur_pct))
print('Missing invocations: {:.2f}%'.format(missing_inv_pct))

# Both have to be below the threshold 
assert (missing_dur_pct <= MAX_MISSING_PCT) & (missing_inv_pct <= MAX_MISSING_PCT)

print('Total rows before cleanup: {}. Dropping rows with missing durations or invocations.' \
  .format(len(app_1.index)))
app_1.drop(app_1[no_dur_idx].index, inplace = True)
app_1.drop(app_1[no_inv_idx].index, inplace = True)
print('Total rows after cleanup: {}.'.format(len(app_1.index)))

# Make sure there are no more NA's in data 
assert not app_1.isnull().any().any()

Missing durations: 0.62%
Missing invocations: 0.25%
Total rows before cleanup: 17687. Dropping rows with missing durations or invocations.
Total rows after cleanup: 17532.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
# Now I'm taking two different paths
# For all the applications where functions take less than minute to execute 
# (vast majority) I'll assume that execution results in resources being allocated 
# for one minute plus pre-defined TTL. This is connservative assumption, but with 
# data resulution of 1 minute it would be difficult to get more granular than this. 
# For these functions I'll construct detailed map of when resources were allocated 
# and when they were free. 
# Remaining, longer functions to be handled separately 
short_app_idx_1 = app_1['duration_ms'] <= ( 60 * 1000 ) 
short_app_1 = app_1[short_app_idx_1]
long_app_1 = app_1[~short_app_idx_1]

print('Total apps: {}, apps with functions shorter than minute: {}, apps with functions longer than minute {}' \
      .format( len(app_1.index), len(short_app_1.index), len(long_app_1.index) )  ) 

Total apps: 17532, apps with functions shorter than minute: 16093, apps with functions longer than minute 1439


##Short Functions 

In [ ]:
# For short functions I'm creating a matrix that represents when resources are allocated. 
# If given funtion is executed at point in time t1 then resources are allocated for 
# next FUN_TTL_MIN minutes plus one minute (max duration of function execution)
# If another function kicks in during this time couning starts from beginning 
# Aggregating with max() but it doesn't really matter - it's a question of having 
# a value vs. not having anything (=0)
short_app_alloc_martix_1 = short_app_1.iloc[:, :1440].rolling(window = FUN_TTL_MIN + 1, axis = 1, min_periods = 1).max()
# Converting to boolean. 
# True - resources are allocated in given one-minute bucket 
# False - resourtces are not allocated 
short_app_alloc_idx_bool_1 = (short_app_alloc_martix_1 > 0)
# Counting allocated buckets per app
short_app_alloc_1 = pd.DataFrame(short_app_alloc_martix_1[short_app_alloc_idx_bool_1].count(axis = 1), \
                                 columns = ['alloc_mins'])
# Calculating percentage of time resources are allocated per app 
short_app_alloc_1['alloc_pct'] = short_app_alloc_1['alloc_mins'] / 1440 * 100 

# Calculating total duration of execution for all the functions alomng with 
# percentage of total time 
# Ignoring anything above 100% - not enough data to calculate scalability impact 
short_app_alloc_1['exec_duration_mins'] = short_app_1['Count'] * short_app_1['duration_ms'] / 1000 / 60 
short_app_alloc_1['exec_duration_pct'] = np.where( short_app_alloc_1['exec_duration_mins'] / 1440 * 100 < 100, \
                                                   short_app_alloc_1['exec_duration_mins'] / 1440 * 100, 100) 

short_app_alloc_1

,alloc_mins,alloc_pct,exec_duration_mins,exec_duration_pct
HashApp,,,,
000481d975e1672df56b11adc7957d13836870d14a1a5e0ab82a6970284b736a,66,4.583,0.383,0.027
000b058a70dedf75e66071d3fe6f8701e31fed235942f4ae734e5d7737a3707f,1440,100.000,44.022,3.057
0016a719d18f743a52142f3b07efd6c17a034fa261e8f445d07da08a0b4bac72,338,23.472,0.796,0.055
0019d10aa8d3ef382c31a7b35d88b487d937d08dcdbdcd3867386807a4f8a8b0,963,66.875,5.032,0.349
001bb40e4afa7a849199f0d584b4005e258b446259e7c066a81f9cd79568f9de,77,5.347,0.161,0.011
...,...,...,...,...
ffdfbb69c91ec3bbc7f9ce02cce783d45d620f4aa050c6e93cf1b7646b0bade8,17,1.181,0.024,0.002
ffe6a32b346421ffd5c745b5206091e692d03a4b71d8dc34aae997ff0e5522c6,512,35.556,2449.469,100.000
fff13120ea625ca705339d9b6dd87747c66d5405135147d44f275e2e55fa348a,11,0.764,0.489,0.034


## Long functions 

In [ ]:
# For short functions for the sake of simplicity I'm assumming that they keep resources 
# allocated all the time. 
# This is a conservative approach and results in having allocation reported as higher 
# than in reality, but the impact on overall results would be well below 10% 
# and calculation with variable-sized windows would be difficult to do with given dataset 
long_app_alloc_1 = pd.DataFrame(long_app_1.index, columns = ['HashApp'])
long_app_alloc_1['alloc_mins'] = 1440
long_app_alloc_1['alloc_pct'] = 100
    
long_app_alloc_1.set_index('HashApp', inplace = True)

# Calculating total duration of execution for all the functions alomng with 
# percentage of total time 
# Ignoring anything above 100% - not enough data to calculate scalability impact 
long_app_alloc_1['exec_duration_mins'] = long_app_1['Count'] * long_app_1['duration_ms'] / 1000 / 60 
long_app_alloc_1['exec_duration_pct'] = np.where( long_app_alloc_1['exec_duration_mins'] / 1440 * 100 < 100, \
                                                  long_app_alloc_1['exec_duration_mins'] / 1440 * 100, 100) 
long_app_alloc_1

,alloc_mins,alloc_pct,exec_duration_mins,exec_duration_pct
HashApp,,,,
009f72f7ae1bc39581c27a9bc53c34ef59d6694bdd09147181251c3ea43caea7,1440,100,1974649.263,100.000
00eaa5b2be78b495d9c0a7f4d43aa5922d317a26605c2c061fe91bd77ce6905f,1440,100,5.001,0.347
01598ea73c90b1d78d06a4763865e0d529976e82d29f0d1501cd2c4bcb76954c,1440,100,5348168.372,100.000
01a3698a425602f90aea6b46b5ff7c4c0637a8de6884ed62bf9d4dea8ee7530c,1440,100,19.999,1.389
01d1a62d3e9fbcebc46c5287cb1c595a78d29fd7d46ff3c7e17d2e4a07447578,1440,100,131.592,9.138
...,...,...,...,...
ff2d5dd04dcabf8ce629c188e5d63d4c1b11962e9ae43d652d5a8758c72dc259,1440,100,4.037,0.280
ff43f8c249c8a39aa05016e9274d46c4e4939dfb7607a215dc31094e0752e25d,1440,100,2826.816,100.000
ff5252ca443b3704f1b180298a2492f5c76c02436ab2a71ce7181f47d95d9209,1440,100,990.497,68.784


In [ ]:
# Joining both datasets back for further analysis 
app_alloc_1 = pd.concat([short_app_alloc_1, long_app_alloc_1])

# Energy usage

## Hardware
First, let's define energy usage of an average CPU core.
This data comes from [Cloud Carbon Footprint](https://www.cloudcarbonfootprint.org/docs/methodology) methodology page, which in turn follows [Etsy's Cloud Jewels](https://codeascraft.com/2020/04/23/cloud-jewels-estimating-kwh-in-the-cloud/)


In [ ]:
AVG_MIN_WATTS = 0.78
AVG_MAX_WATTS = 3.76

## FaaS energy usage

In [ ]:
# Energy consumption of workload deployed on FaaS
# It consists of enegry consumed when resources are allocated but not used:
app_alloc_1['idle_kwh'] = app_alloc_1['alloc_mins'] / 60 * AVG_MIN_WATTS / 1000

# .. and enegry consumed when resources are actually used 
# first part used to calculate number of hours within a day when function is running
# second part represent enegry consumption assumming function is running at given CPU percentage 
app_alloc_1['active_kwh'] = app_alloc_1['exec_duration_pct']/100 * 1440 / 60 * \
  (AVG_MAX_WATTS - AVG_MIN_WATTS) * ACTIVE_FUNCTION_CPU_PCT/100 \
  / 1000 # to convert to kW

app_alloc_1.sum()

alloc_mins            14368935.000
alloc_pct               997842.708
exec_duration_mins   283611671.215
exec_duration_pct       183997.900
idle_kwh                   186.796
active_kwh                  65.798
dtype: float64

##VM enegry usage

In [ ]:
vm_alloc = pd.DataFrame(app_alloc_1.index)
vm_alloc.set_index('HashApp', inplace = True)

no_of_cpu = None
no_of_servers = None

# Configuration #1
# One app deployed on one-CPU VM running all the time 
no_of_cpu = 1
no_of_servers = 1
vm_alloc['conf_1_idle_kwh'] = AVG_MIN_WATTS/1000 * 24 * no_of_cpu * no_of_servers
# Active consumption is the same as for serverless 
vm_alloc['conf_1_active_kwh'] = app_alloc_1['active_kwh']

# Configuration #2
# One app deployed on two-CPU VM running all the time 
no_of_cpu = 2
no_of_servers = 1
vm_alloc['conf_2_idle_kwh'] = AVG_MIN_WATTS/1000 * 24 * no_of_cpu * no_of_servers
# Active consumption is the same as for serverless 
vm_alloc['conf_2_active_kwh'] = app_alloc_1['active_kwh']

# Configuration #3
# One app deployed on four-CPU VM running all the time 
no_of_cpu = 4
no_of_servers = 1
vm_alloc['conf_3_idle_kwh'] = AVG_MIN_WATTS/1000 * 24 * no_of_cpu * no_of_servers
# Active consumption is the same as for serverless 
vm_alloc['conf_3_active_kwh'] = app_alloc_1['active_kwh']

# Configuration #4
# One app deployed on four-CPU VM running all the time plus warm standby 
no_of_cpu = 4
no_of_servers = 2
vm_alloc['conf_4_idle_kwh'] = AVG_MIN_WATTS/1000 * 24 * no_of_cpu * no_of_servers
# Active consumption is the same as for serverless 
vm_alloc['conf_4_active_kwh'] = app_alloc_1['active_kwh']

vm_alloc.sum()


conf_1_idle_kwh      328.199
conf_1_active_kwh     65.798
conf_2_idle_kwh      656.398
conf_2_active_kwh     65.798
conf_3_idle_kwh     1312.796
conf_3_active_kwh     65.798
conf_4_idle_kwh     2625.592
conf_4_active_kwh     65.798
dtype: float64